## Preprocessing data of Domcke et al., Science (2020)

#### Data from https://descartes.brotmanbaty.org/bbi/human-chromatin-during-development/
##### Cicero_coaccessibility_scores_by_cell_type.csv.gz

In [ ]:
import os
import pandas as pd
from pybedtools import BedTool

domcke_cre = pd.read_csv("Cicero_coaccessibility_scores_by_cell_type.csv.gz", sep=",")

In [ ]:
cerebrum = [x for x in domcke_cre.columns if "cerebrum" in x]
cerebrum_cre = domcke_cre.loc[:, ["Peak1","Peak2"]+cerebrum]

In [ ]:
cerebrum_cre['is_cerebrum'] = cerebrum_cre.iloc[:, 2:].notna().sum(axis=1)
cerebrum_cre = cerebrum_cre.loc[cerebrum_cre['is_cerebrum'] != 0]

In [ ]:
outdir = "/data1/cwas-plus/functional_annotations/Domcke"

domcke = cerebrum_cre

In [ ]:
for c in domcke.columns.tolist()[2:]:
    if not "SKOR2" in c:
        c_peaks = domcke.loc[(domcke[c].notna())]
        c_peaks_bed = pd.concat([c_peaks['Peak1'], c_peaks['Peak2']]).drop_duplicates().str.split("-", expand=True)
        c_peaks_bed = BedTool.from_dataframe(c_peaks_bed).sort().to_dataframe()
        c_rename = c.split("_")[0].replace(" ", "_")
        c_peaks_bed.to_csv(os.path.join(outdir, "Domcke_"+c_rename+"_co-acessible_peaks.bed"), sep='\t', index=False, header=False)
        
        print(c_rename, len(c_peaks_bed))
    